In [ ]:
# Don't need to run this cell
import os
os.chdir("/content/drive/MyDrive/ColabNotebooks")

# For Colab User 

* set `runtime=GPU` then start to run

In [ ]:
# For Colab: Install FARM
!pip install torch==1.6.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html
!pip install farm==0.5.0
!pip install -U -q emoji soynlp
!git clone https://github.com/e9t/nsmc

전처리하기

In [16]:
import os
import re
import emoji
import pandas as pd
from pathlib import Path
from soynlp.normalizer import repeat_normalize

def read_data(path:str, header=None):
    return pd.read_csv(path, sep='\t', header=header)

def clean(x):
    emojis = ''.join(emoji.UNICODE_EMOJI.keys())
    pattern = re.compile(f'[^ .,?!/@$%~％·∼()\x00-\x7Fㄱ-힣{emojis}]+')
    url_pattern = re.compile(
        r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)')
    
    x = pattern.sub(' ', x)
    x = url_pattern.sub('', x)
    x = x.strip()
    x = repeat_normalize(x, num_repeats=2)
    return x

def preprocess_dataframe(df:pd.DataFrame):
    r"""
    Changed the code
    source from: https://colab.research.google.com/drive/1IPkZo1Wd-DghIOK6gJpcb0Dv4_Gv2kXB
    """

    label_dict = {0:"bad", 1:"good"}
    df['document'] = df['document'].apply(lambda x: clean(str(x)))
    df['label'] = df['label'].apply(label_dict.get)
    return df

df_train = preprocess_dataframe(read_data("./nsmc/ratings_train.txt", header=0))
df_test = preprocess_dataframe(read_data("./nsmc/ratings_test.txt", header=0))
df_train.loc[:, ["label", "document"]].to_csv("./nsmc/train.tsv", sep="\t", index=False)
df_test.loc[:, ["label", "document"]].to_csv("./nsmc/test.tsv", sep="\t", index=False)

In [17]:
!ls nsmc

code		  ratings_train.txt  raw	synopses.json  train.tsv
ratings_test.txt  ratings.txt	     README.md	test.tsv


In [4]:
import sys
import torch
from pathlib import Path
from farm.modeling.tokenization import Tokenizer
from farm.data_handler.processor import TextClassificationProcessor
from farm.data_handler.data_silo import DataSilo
from farm.modeling.language_model import LanguageModel
from farm.modeling.prediction_head import TextClassificationHead
from farm.modeling.adaptive_model import AdaptiveModel
from farm.modeling.optimization import initialize_optimizer
from farm.train import Trainer
from farm.utils import MLFlowLogger

repo_path = Path() # Path().absolute().parent
sys.path.append(str(repo_path))

# Change to your experiment name and run name
EXP_NAME = "FARM_tutorial"
RUN_NAME = "NSMC_colab"
ml_logger = MLFlowLogger(tracking_uri="https://public-mlflow.deepset.ai/")
ml_logger.init_experiment(experiment_name=EXP_NAME, run_name=RUN_NAME)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Devices available: {}".format(device))

03/29/2021 09:45:11 - INFO - farm.modeling.prediction_head -   Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .



 __          __  _                            _        
 \ \        / / | |                          | |       
  \ \  /\  / /__| | ___ ___  _ __ ___   ___  | |_ ___  
   \ \/  \/ / _ \ |/ __/ _ \| '_ ` _ \ / _ \ | __/ _ \ 
    \  /\  /  __/ | (_| (_) | | | | | |  __/ | || (_) |
     \/  \/ \___|_|\___\___/|_| |_| |_|\___|  \__\___/ 
  ______      _____  __  __  
 |  ____/\   |  __ \|  \/  |              _.-^-._    .--.
 | |__ /  \  | |__) | \  / |           .-'   _   '-. |__|
 |  __/ /\ \ |  _  /| |\/| |          /     |_|     \|  |
 | | / ____ \| | \ \| |  | |         /               \  |
 |_|/_/    \_\_|  \_\_|  |_|        /|     _____     |\ |
                                     |    |==|==|    |  |
|---||---|---|---|---|---|---|---|---|    |--|--|    |  |
|---||---|---|---|---|---|---|---|---|    |==|==|    |  |
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
 
Devices available: cuda


<center><img src="https://drive.google.com/uc?id=1hbtUClFoXg45IbViZoFRLnnDGVlr9Dlb" alt="Fine-tuning" width="30%" height="30%"></center>

# FARM

> Framework for Adapting Representation Models

이 패키지를 한 마디로 요약하면 Fine-tuning에 최적화된 도구다.

최근의 자연어처리 분야는 Transformer와 그 변형의 등장으로 인해, 보통 2단계로 나눠서 학습이 진행된다. 
1. **Pretrained Language Modeling**

   대량의 텍스트 데이터를 이용해 비지도학습(unsupervised learning)으로 언어 모델링은 진행한다. 언어 모델링이란 인간의 언어를 컴퓨터로 모델링하는 과정이다. 쉽게 말하면, 모델에게 단어들을 입력했을 때, 제일 말이 되는 단어(토큰)을 뱉어내게 하는 것이다. 과거에는 단어(토큰)의 순서가 중요했었다. 즉, 일정 단어들의 시퀀스 $x_{1:t-1}$가 주어지면, $t$번째 단어인 $x_t$를 잘 학습시키는 것이었다. 이를 Auto Regressive Modeling이라고도 한다. 그러나, Masked Language Modeling 방법이 등장했는데, 이는 랜덤으로 맞춰야할 단어를 가린 다음에 가려진 단어 $x_{mask}$가 포함된 시퀀스 $x_{1:t}$ 를 모델에게 입력하여 맞추는 학습 방법이다. 이러한 방법이 좋은 성과를 거두면서, 최근에는 모든 
2. **Fine-tuning**

    PLM(Pretrained Language Model)을 만들고 나면, 각기 다른 downstream task에 따라서 fine-tuning을 하게 된다. Downstream task은 구체적으로 풀고 싶은 문제를 말하며, 주로 다음과 같은 문제들이다.
    * 텍스트 분류 Text Classification - 예시: 영화 댓글 긍정/부정 분류하기
    * 개체명인식 NER(Named Entity Recognition) - 예시: 특정 기관명, 인명 및 시간 날짜 등 토큰에 알맞는 태그로 분류하기
    * 질의응답 Question and Answering - 예시: 특정 지문과 질의(query)가 주어지면 대답하기

오늘 소개할 FARM 패키지는 2번째 단계인 Fine-tuning을 보다 손쉽게 만들어놓은 패키지다. Github에서 Colab tutorial과 함께 보면 좋다.

- **Tutorial github:** https://github.com/simonjisu/FARM_tutorial
- **Colab Tutorial:** [링크](https://colab.research.google.com/github/simonjisu/FARM_tutorial/blob/main/notebooks/FARM_colab.ipynb)


## Core Features

- **Easy fine-tuning of language models** to your task and domain language
- **Speed**: AMP(Automatic Mixed Precision) optimizers (~35% faster) and parallel preprocessing (16 CPU cores => ~16x faster)
- **Modular design** of language models and prediction heads
- Switch between heads or combine them for **multitask learning**
- **Full Compatibility** with HuggingFace Transformers' models and model hub
- **Smooth upgrading** to newer language models
- Integration of **custom datasets** via Processor class
- Powerful **experiment tracking** & execution
- **Checkpointing & Caching** to resume training and reduce costs with spot instances
- Simple **deployment** and **visualization** to showcase your model

<details>
<summary> 👉 what is AMP? </summary>

**Reference**
- https://github.com/NVIDIA/apex
- https://forums.fast.ai/t/mixed-precision-training/20720

**mixed precision training이란**
- 처리 속도를 높이기 위한 FP16(16bit floating point)연산과 정확도 유지를 위한 FP32 연산을 섞어 학습하는 방법
- Tensor Core를 활용한 FP16연산을 이용하면 FP32연산 대비 절반의 메모리 사용량과 8배의 연산 처리량 & 2배의 메모리 처리량 효과가 있다
</details>


---

# NSMC 데이터로 FARM 알아보기

## NSMC 데이터

NSMC(Naver Sentiment Movie Corpus)는 한국어로 된 영화 댓글 데이터 세트다. 해당 Task는 타겟 값이 긍정(1)/부정(0)이 되는 Binary Text Classification 문제로 볼 수 있다. https://github.com/e9t/nsmc 에서 받을 수 있다.

<center><img src="https://drive.google.com/uc?id=1FIGIBtZxtuKD5Prps5vPOPldBb0xHwzH" alt="Fine-tuning" width="50%" height="50%" align="center"></center>

## Fine-tuning Process

<center><img src="https://drive.google.com/uc?id=1j9pn8Lpg7sy6S8Ubvq3E7JLWf28KvRt4" alt="Fine-tuning" width="50%" height="50%" align="center"></center>

Fine-tuning Process는 위 그림과 같이 진행된다.

* Load Data: 데이터를 알맞는 형식(json, csv 등)으로 불러온다.
* Create Dataset: 데이터세트(Dataset) 만들기
    * Tokenization: 텍스트를 토큰으로 나누고, 단어장(vocab)을 생성한다.
    * ToTensor: vocab에 해당하는 단어를 수치화하는 과정 (`input_ids` in transformers)
    * Attention Mask: 패딩계산을 피하기 위해 Attention 해야할 토큰만 masking(`attention_mask` in transformers)
* Create Dataloader: 훈련, 평가시 배치크기 단위로 데이터를 불러오는 객체
* Create Model:
    * Pretrained Language Model: 대량의 텍스트 데이터로 사전에 훈련된 모델 
$$\underset{\theta}{\arg \max} P(x_{mask} \vert x_{1:t})$$
    * Fine-tuninig Layer: Downstream Task에 맞춰서 학습한다.       
$$\underset{\theta}{\arg \max}P(y\vert x_{1:t})$$
        예를 들어, 영화 긍정/부정 분류 문제의 경우
$$\underset{\theta}{\arg \max} P(y=\text{긍정/부정} \vert x_{1:t})$$
* Train Model: 모델 훈련
* Eval Model: 모델 평가
* Inference: 모델 서비스

## Processor & Data Silo

<center><img src="https://drive.google.com/uc?id=1XCc0AJpPBMFcC81NW0A6w0mpswZ2KU7h" alt="Fine-tuning" width="60%" height="50%" align="center"></center>

* **Processor**는 file 혹은 request를 PyTorch Datset로 만들어 주는 역할이다. 자세한 인자값은 다음 코드 블록에서 설명한다.
* **Data Silo**는 train, dev, test sets를 관리하고, Processor의 function들 이용해 각 set를 DataLoader로 변환한다.
* **Processor**는 각 데이터를 처리할 때, **Samples**, **SampleBasket**에 담게 되는데, 이들은 raw document를 관리하는 객체이며 tokenized, features등 데이터를 저장하고 있다. 이렇게 하는 이유는 하나의 소스 텍스트(raw text)에서 여러개의 샘플을 생성할 수도 있기 때문이다(e.g. QA task)
    ```python
    def dataset_from_dicts(self, ...)
        # ...
        for dictionary, input_ids, segment_ids, padding_mask, tokens in zip(
                dicts, input_ids_batch, segment_ids_batch, padding_masks_batch, tokens_batch
        ):
            # ...
            # Add Basket to self.baskets
            curr_sample = Sample(
                id=None,
                clear_text=dictionary,
                tokenized=tokenized,
                features=[feat_dict]
            )
            curr_basket = SampleBasket(
                id_internal=None,
                raw=dictionary,
                id_external=None,
                samples=[curr_sample]
               )
            self.baskets.append(curr_basket)

        # ...
    ```

사용하는 방법은 다음과 같다.

In [6]:
DATA_PATH = repo_path / "nsmc"
PRETRAINED_MODEL_NAME_OR_PATH = "beomi/kcbert-base"  # Reference: https://github.com/Beomi/KcBERT
MAX_LENGTH = 150
LABEL_LIST = ["bad", "good"]
TRAIN_FILE = "train.tsv"
TEST_FILE = "test.tsv"
TASK_TYPE = "text_classification"

tokenizer = Tokenizer.load(
    pretrained_model_name_or_path=PRETRAINED_MODEL_NAME_OR_PATH,
    do_lower_case=False,
)

processor = TextClassificationProcessor(
    tokenizer=tokenizer,  # tokenizer 
    train_filename=TRAIN_FILE,  # training data 파일명
    dev_filename=None,  # development data 파일명, 없으면, dev_split 비율만큼 training data에서 자른다 
    test_filename=TEST_FILE,  # test data 파일명
    dev_split=0.1,  # development data로 설정할 비율
    header=0,  # csv, tsv, excel 등 tabular형태 데이터에서 첫행(보통은 컬럼명)의 위치
    max_seq_len=MAX_LENGTH,  # 문장의 최대 길이
    data_dir=str(DATA_PATH),  # 데이터의 디렉토리
    label_list=LABEL_LIST,  # 레이블 리스트(string 필요)
    metric="acc",  # 평가지표
    label_column_name="label",  # tabular형태 데이터에서 레이블의 컬럼명
    text_column_name="document",  # tabular형태 데이터에서 텍스트의 컬럼명
    delimiter="\t"
)


data_silo = DataSilo(
    processor=processor,
    batch_size=8,
    eval_batch_size=8,
    caching=True
)

03/29/2021 09:45:12 - INFO - farm.modeling.tokenization -   Loading tokenizer of type 'BertTokenizer'
03/29/2021 09:45:14 - INFO - farm.data_handler.data_silo -   
Loading data into the data silo ... 
              ______
               |o  |   !
   __          |:`_|---'-.
  |__|______.-/ _ \-----.|       
 (o)(o)------'\ _ /     ( )      
 
03/29/2021 09:45:14 - INFO - farm.data_handler.data_silo -   Loading train set from: nsmc/train.tsv 
03/29/2021 09:45:15 - INFO - farm.data_handler.data_silo -   Got ya 1 parallel workers to convert 149539 dictionaries to pytorch datasets (chunksize = 2000)...
03/29/2021 09:45:15 - INFO - farm.data_handler.data_silo -    0 
03/29/2021 09:45:15 - INFO - farm.data_handler.data_silo -   /|\
03/29/2021 09:45:15 - INFO - farm.data_handler.data_silo -   /'\
03/29/2021 09:45:15 - INFO - farm.data_handler.data_silo -   
Preprocessing Dataset nsmc/train.tsv:   0%|          | 0/149539 [00:00<?, ? Dicts/s]/usr/local/lib/python3.7/dist-packages/transformers/to

데이터는 다음과 같이 tokenization 되며, sample 객체에 저장된다. 

<center><img src="https://drive.google.com/uc?id=1DVPT_Rjv_SI4ggJZzqfPh0MgsMa1Q9El" alt="Fine-tuning" width="100%" height="50%" align="center"></center>

```plaintext
03/28/2021 22:12:15 - INFO - farm.data_handler.processor -   

      .--.        _____                       _      
    .'_\/_'.     / ____|                     | |     
    '. /\ .'    | (___   __ _ _ __ ___  _ __ | | ___ 
      "||"       \___ \ / _` | '_ ` _ \| '_ \| |/ _ \ 
       || /\     ____) | (_| | | | | | | |_) | |  __/
    /\ ||//\)   |_____/ \__,_|_| |_| |_| .__/|_|\___|
   (/\||/                             |_|           
______\||/___________________________________________                     

ID: 437-0
Clear Text: 
 	text_classification_label: good
 	text: 이 영화를 보고 두통이 나았습니다. ㅠ ㅠ
Tokenized: 
 	tokens: ['이', '영화를', '보고', '두', '##통이', '나', '##았습니다', '.', '[UNK]', '[UNK]']
 	offsets: [0, 2, 6, 9, 10, 13, 14, 18, 20, 22]
 	start_of_word: [True, True, True, True, False, True, False, False, True, True]
Features: 
 	input_ids: [2, 2451, 25833, 8198, 917, 11765, 587, 21809, 17, 1, 1, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 	padding_mask: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 	segment_ids: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 	text_classification_label_ids: [1]
```

## Modeling Layers: AdaptiveModel = LanguageModel + PredictionHead

<center><img src="https://drive.google.com/uc?id=1OLWdr8rh7ucpF9t55gzVeMawMBJbRiEC" alt="Fine-tuning" width="60%" height="50%" align="center"></center>

* **LanguageModel**은 pretrained language models(BERT, XLNet ...)의 표준 클래스 
* **PredictionHead**는 모든 down-stream tasks(NER, Text classification, QA ...)를 표준 클래스
* **AdaptiveModel**은 위 두 가지 모들의 결합, 하나의 LanguageModel과 여러 개의 PredictionHead를 결합할 수 있다.

In [ ]:
# LanguageModel: Build pretrained language model
EMBEDS_DROPOUT_PROB = 0.1
TASK_NAME = "text_classification"

language_model = LanguageModel.load(PRETRAINED_MODEL_NAME_OR_PATH, language="korean")
# PredictionHead: Build predictor layer
prediction_head = TextClassificationHead(
    num_labels=len(LABEL_LIST), 
    class_weights=data_silo.calculate_class_weights(
        task_name=TASK_NAME
    )
)
model = AdaptiveModel(
    language_model=language_model,
    prediction_heads=[prediction_head],
    embeds_dropout_prob=EMBEDS_DROPOUT_PROB,
    lm_output_types=["per_sequence"],
    device=device
)

In [41]:
print(f"Model: {type(model)}")
for k, v in model.named_children():
    for k1, v1 in v.named_children():
        
        for k2, v2 in v1.named_children():
            print("----------------------------"*2)
            print(f"Module: {k} | Layer: {k2}")
            print("----------------------------"*2)
            if k2 == "encoder":
                print("Showing last layer")
                print(list(v2.children())[0][-1])
                break
            else:
                print(v2)

print("Last Dropout Layer")
print(model.dropout)

Model: <class 'farm.modeling.adaptive_model.AdaptiveModel'>
--------------------------------------------------------
Module: language_model | Layer: embeddings
--------------------------------------------------------
BertEmbeddings(
  (word_embeddings): Embedding(30000, 768, padding_idx=0)
  (position_embeddings): Embedding(300, 768)
  (token_type_embeddings): Embedding(2, 768)
  (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
  (dropout): Dropout(p=0.1, inplace=False)
)
--------------------------------------------------------
Module: language_model | Layer: encoder
--------------------------------------------------------
Showing last layer
BertLayer(
  (attention): BertAttention(
    (self): BertSelfAttention(
      (query): Linear(in_features=768, out_features=768, bias=True)
      (key): Linear(in_features=768, out_features=768, bias=True)
      (value): Linear(in_features=768, out_features=768, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (

실제로 transformers 패키지와 비교해보면 비슷하다.

In [44]:
from transformers import BertForSequenceClassification
bert = BertForSequenceClassification.from_pretrained(PRETRAINED_MODEL_NAME_OR_PATH)
print("----------------------------"*2)
print(f"Module: classifer")
print("----------------------------"*2)
print(bert.classifier)
print("----------------------------"*2)
print(f"Module: dropout")
print("----------------------------"*2)
print(bert.dropout)

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initiali

--------------------------------------------------------
Module: classifer
--------------------------------------------------------
Linear(in_features=768, out_features=2, bias=True)
--------------------------------------------------------
Module: dropout
--------------------------------------------------------
Dropout(p=0.1, inplace=False)


## Train & Eval & Inference

<center><img src="https://drive.google.com/uc?id=1bD54igqAn7T96gDCFZ2uxzFHpZIL5GOh" alt="Fine-tuning" width="60%" height="50%" align="center"></center>

### Train & Eval

In [46]:
LEARNING_RATE = 2e-5
N_EPOCHS = 1
N_GPU = 1
checkpoint_path = "./ckpt/NSMC"

# Initialize Optimizer
model, optimizer, lr_schedule = initialize_optimizer(
    model=model,
    device=device,
    learning_rate=LEARNING_RATE,
    n_batches=len(data_silo.loaders["train"]),
    n_epochs=N_EPOCHS
)
# EarlyStopping
earlymetric = "f1" if args.task_name == "question_answering" else "acc" 
mode = "max" if args.task_name in ["text_classification", "question_answering"] else "min"
earlystop = EarlyStopping(
    save_dir=checkpoint_path,
    metric=earlymetric,
    mode=mode,
    patience=3,
)

# Trainer
trainer = Trainer(
    model=model,
    optimizer=optimizer,
    lr_schedule=lr_schedule,
    data_silo=data_silo,
    early_stopping=earlystop,
    evaluate_every=20,
    checkpoints_to_keep=3,
    checkpoint_root_dir=checkpoint_path,
    checkpoint_every=200,
    epochs=N_EPOCHS,
    n_gpu=N_GPU,
    device=device, 
)
# now train!
model = trainer.train()

03/29/2021 10:52:16 - INFO - farm.modeling.optimization -   Loading optimizer `TransformersAdamW`: '{'correct_bias': False, 'weight_decay': 0.01, 'lr': 2e-05}'
03/29/2021 10:52:16 - INFO - farm.modeling.optimization -   Using scheduler 'get_linear_schedule_with_warmup'
03/29/2021 10:52:16 - INFO - farm.modeling.optimization -   Loading schedule `get_linear_schedule_with_warmup`: '{'num_warmup_steps': 1674.7, 'num_training_steps': 16747}'
03/29/2021 10:52:16 - INFO - farm.train -   
 

          &&& &&  & &&             _____                   _             
      && &\/&\|& ()|/ @, &&       / ____|                 (_)            
      &\/(/&/&||/& /_/)_&/_&     | |  __ _ __ _____      ___ _ __   __ _ 
   &() &\/&|()|/&\/ '%" & ()     | | |_ | '__/ _ \ \ /\ / / | '_ \ / _` |
  &_\_&&_\ |& |&&/&__%_/_& &&    | |__| | | | (_) \ V  V /| | | | | (_| |
&&   && & &| &| /& & % ()& /&&    \_____|_|  \___/ \_/\_/ |_|_| |_|\__, |
 ()&_---()&\&\|&&-&&--%---()~                                     

KeyboardInterrupt: ignored

훈련 과정에 계속 Log가 찍히고, Processor단계에서 입력해둔 `test_filename`로 평가도 해준다. 다음 그림은 430개의 배치 데이터(batch_size=256)를 돌렸을 때 earlystopping한 결과다.

<center><img src="https://drive.google.com/uc?id=1m1K9CjBNulC4dzSxC1vKjLb94p9BQu26" alt="Fine-tuning" width="80%" height="50%" align="center"></center>

### Inference

In [ ]:
import os
os.chdir("/content/drive/MyDrive/ColabNotebooks")

In [23]:
from farm.infer import Inferencer
from pprint import PrettyPrinter

basic_texts = [
    {"text": "기생충... 이 영화 정말 올해의 대작임."},
    {"text": "황정민 나오는 영화는 다 볼만한듯."},
]

infer_model = Inferencer.load(
    model_name_or_path="./ckpt/best_nsmc",
    task_type="text_classification"
)
result = infer_model.inference_from_dicts(dicts=basic_texts)
print()

03/29/2021 11:39:09 - INFO - farm.utils -   device: cpu n_gpu: 0, distributed training: False, automatic mixed precision training: None
03/29/2021 11:39:11 - INFO - farm.modeling.adaptive_model -   Found files for loading 1 prediction heads
03/29/2021 11:39:11 - WARNING - farm.modeling.prediction_head -   `layer_dims` will be deprecated in future releases
03/29/2021 11:39:11 - INFO - farm.modeling.prediction_head -   Prediction head initialized with size [768, 2]
03/29/2021 11:39:11 - INFO - farm.modeling.prediction_head -   Using class weights for task 'text_classification': [0.9966925978660583, 1.0033293962478638]
03/29/2021 11:39:11 - INFO - farm.modeling.prediction_head -   Loading prediction head from ckpt/NSMC/prediction_head_0.bin
03/29/2021 11:39:12 - INFO - farm.modeling.tokenization -   Loading tokenizer of type 'BertTokenizer'
03/29/2021 11:39:12 - INFO - farm.data_handler.processor -   Initialized processor without tasks. Supply `metric` and `label_list` to the constructor 

In [24]:
PrettyPrinter().pprint(result)

[{'predictions': [{'context': '기생충,,, 이 영화 정말 재밌네요.',
                   'end': None,
                   'label': 'good',
                   'probability': 0.83329886,
                   'start': None},
                  {'context': '황정민 나오는 영화는 다 볼만한듯.',
                   'end': None,
                   'label': 'good',
                   'probability': 0.7448745,
                   'start': None}],
  'task': 'text_classification'}]


---

# MLflow

MLflow를 이용하 빠르고 쉽게 실험을 관리하고, 관련 평가지표도 함께 볼 수 있다. public mlflow([링크](https://public-mlflow.deepset.ai/#/experiments/313/runs/05e7e3d4945642f9ab3e296637d57c26))에서 확인하기

<center><img src="https://drive.google.com/uc?id=13Cg8eziHBgA3JLwZJ3Bo8YzeySWPRmiP" alt="Fine-tuning" width="30%" height="50%" align="center"></center>

Train과 Dev 세트의 loss는 다음과 같다.

<center><img src="https://drive.google.com/uc?id=1cpFWVvjkSqshvN0hS_CuPk4RjyEyM0AV" alt="Fine-tuning" width="90%" height="50%" align="center"></center>

Dev 세트의 정확도는 다음과 같다.

<center><img src="https://drive.google.com/uc?id=1VPso9Gx60V8_dgE4as054n7kymCoQ9w5" alt="Fine-tuning" width="90%" height="50%" align="center"></center>

# TASK Supported

|Task|BERT|RoBERTa*|XLNet|ALBERT|DistilBERT|XLMRoBERTa|ELECTRA|MiniLM|
|---|---|---|---|---|---|---|---|---|
|Text classification|x|x|x|x|x|x|x|x|
|NER|x|x|x|x|x|x|x|x|
|Question Answering|x|x|x|x|x|x|x|x|
|Language Model Fine-tuning|x||||||||
|Text Regression|x|x|x|x|x|x|x|x|
|Multilabel Text classif.|x|x|x|x|x|x|x|x|
|Extracting embeddings|x|x|x|x|x|x|x|x|
|LM from scratch|x||||||||
|Text Pair Classification|x|x|x|x|x|x|x|x|
|Passage Ranking|x|x|x|x|x|x|x|x|
|Document retrieval (DPR)|x|x||x|x|x|x|x|

# Compare to others

<center><img src="https://drive.google.com/uc?id=1TZoRpza8-o4wSTr0s16f8hHQRroLQg30" alt="Fine-tuning" width="90%" height="50%" align="center"></center>

다른 모델과 비교해보면 FARM은 조금 더 huggingface와 pytorch-lightning의 합본 축약 버전이라고 생각할 수 있다. 마치 Tensorflow v1과 keras의 차이 느낌이다.

## FARM 장단점

장점:

* 데이터 세트만 준비되어 있으면, 다른 패키지에 비해 상대적으로 설정 할 것이 적음
* 훈련 속도가 빠르고, 실험 기록 및 관리이 편리해서 빠르게 실험해 볼 수 있다.
* 텐서보드 대신 mlflow 사용 가능

단점: 

* customization이 상대적으로 힘듦
* 아직 발전 중이라 불안정하고 documentaton이 잘 안되어 있다.